# Lector y graficador de archivo de salida de StarLight

### Lector 

Esta primera parte del programa lee un unico archivo de salida de StarLight, separando los datos que nos son utiles y eligiendo a la poblacion que aporto mas del 4% al espectro resultante

In [16]:
import numpy as np
import pandas as pd

In [17]:
nombre_archivo = input(
    "Hola, ¿Cual es el nombre del archivo? (Aclarar extención): "
)

Hola, ¿Cual es el nombre del archivo? (Aclarar extención):  SL573_new.out


In [18]:
lineas = []
df = pd.DataFrame(
    columns=["Valor", "Parametro"]
)  # en esta tabla voy a guardar los resultados de la sintesis

with open(nombre_archivo, "r") as archivo:
    for linea in archivo:
        l = linea.strip().split()
        lineas.append(l)
        # -------------------------------------------------------------------------------------------------------------------
        # ACA GUARDO LOS RESULTADOS DE LA SISTESIS QUE ME INTERESAN (ESTA PARTE HABRIA Q OPTIMIZARLA MEJOR)
        # -------------------------------------------------------------------------------------------------------------------
        if l and l[-1] == "[chi2/Nl_eff]":
            df.loc[0] = [
                float(l[0]),
                l[-1],
            ]  # aca pongo el float porque si no lo guarda como 'str'

        if len(l) >= 3 and l[-2] == "[AV_min" and l[-1] == "(mag)]":
            df.loc[1] = [float(l[0]), l[-2] + l[-1]]

        if len(l) >= 3 and l[-2] == "[v0_min" and l[-1] == "(km/s)]":
            df.loc[2] = [float(l[0]), l[-2] + l[-1]]

        if len(l) >= 3 and l[-2] == "[vd_min" and l[-1] == "(km/s)]":
            df.loc[3] = [float(l[0]), l[-2] + l[-1]]

        # -------------------------------------------------------------------------------------------------------------------
        # ACA LEO EL TAMAÑO DE LA BASE
        # -------------------------------------------------------------------------------------------------------------------
        if l and l[-1] == "[N_base]":
            d = int(
                l[0]
            )  # cuando me lo lee del archivo lo hace como ''str' entonces lo paso a entero
        # -------------------------------------------------------------------------------------------------------------------
        # ACA LEO la cantidad de puntos que tengo en longitud de onda
        # -------------------------------------------------------------------------------------------------------------------
        if l and l[-1] == "[Nl_obs]":
            Nl_obs = int(
                l[0]
            )  # cuando me lo lee del archivo lo hace como ''str' entonces lo paso a entero
    # -------------------------------------------------------------------------------------------------------------------
    # ACA CREO EL DATAFRAME DE LA BASE
    # -------------------------------------------------------------------------------------------------------------------
    del lineas[62][
        0
    ]  # aca borro el primer elemento de esa linea('#') para usarla como columna de la tabla
    df1_str = pd.DataFrame(columns=lineas[62])
    for i in range(d):
        df1_str.loc[i] = lineas[i + 63]
    # -------------------------------------------------------------------------------------------------------------------
    # ACA CREO EL DATAFRAME DE LOS DATOS OBSERVADOS Y SINTETICOS
    # -------------------------------------------------------------------------------------------------------------------
    df2_str = pd.DataFrame(columns=["l_obs", "f_obs", "f_syn", "wei"])
    for i in range(Nl_obs):
        df2_str.loc[i] = lineas[
            i + d * 3 + 81
        ]  # el d*3 es por la cantidad de veces que aparece la base en el medio
    # y el 81 es por los primeros 63 lugares y los espacios entre tabla y tabla
# -------------------------------------------------------------------------------------------------------------------
# ACA PASO LOS ELEMNTOS DE LOS DATAFRAMES DE 'STR' A 'FLOAT' DADO QUE AL LEERLOS ASI ME LOS TOMA COMO TEXTO
# -------------------------------------------------------------------------------------------------------------------
df1 = df1_str.drop("component_j", axis=1).astype(
    float
)  # para este es mas complejo porque tengo que dajar 1 columna como 'str'
df1["component_j"] = df1_str[
    "component_j"
]  # defino una nueva columna de df1 para guardar la columna que era texto del anterior y que tire
df2 = df2_str.astype(float)
# -------------------------------------------------------------------------------------------------------------------
# ACA SOLO VOY A TOMAR LAS POBLACIONES QUE APORTEN MAS DEL 4% AL ESPECTRO RESULTANTE
# -------------------------------------------------------------------------------------------------------------------
df1 = df1[df1["x_j(%)"] > 4].reset_index(
    drop=True
)  # aca solo me quedo con los que tienen un aporde mayor a 4%
# -------------------------------------------------------------------------------------------------------------------
# ACA REESCALEO EL APORTE PARA QUE LA SUMA DE TODAS LAS POBLACIONES CON APORTE MAYOR AL 4% SEA EL 100%
# -------------------------------------------------------------------------------------------------------------------
df1["x_j(%)"] = df1["x_j(%)"] * 100 / df1["x_j(%)"].sum()

In [19]:
# df

In [20]:
# df1

### Determinación de Edad y Metalicidad

Para determinar la Edad y Metalicidad del espectro sintetico resultante lo que hacemos es utilizar el aporte de cada una de las SSP que esten por encima de 4% y usar el porcentaje de su aporte para calcular un promedio del logaritmo de la edad y metalicidad pesado por este aporte. 

In [21]:
edad = int(((df1["x_j(%)"] * df1["age_j(yr)"])).sum() / df1["x_j(%)"].sum())
edad = int(
    10
    ** (
        ((df1["x_j(%)"] * np.log10(df1["age_j(yr)"]))).sum()
        / df1["x_j(%)"].sum()
    )
)
edad = f"{edad:.3e}"  # lo pongo en notacion cientifica con 3 decimales despues de la coma
print(edad)

5.602e+08


In [22]:
met = ((df1["x_j(%)"] * df1["Z_j"])).sum() / df1["x_j(%)"].sum()
met = round(met, 5)  # redondeo la metalicidad a 5 decimales
print(met)

0.006


In [23]:
edad_1 = int(
    10
    ** (
        ((df1["x_j(%)"] * np.log10(df1["age_j(yr)"]))).sum()
        / df1["x_j(%)"].sum()
    )
)
edad_1 = np.log10(edad_1)
edad_1 = round(edad_1, 2)
edad_1 = f"{edad_1:.2e}"
print(edad_1)

8.75e+00


### Graficador 

En esta parte del programa vamos a graficar los datos leidos anteriormente de StarLight utilizando la libreria Plotly.

In [9]:
import plotly.graph_objects as go
import plotly.offline as pyo

In [10]:
fig = go.Figure()
# ------------------------------------------------------------------------------------------------------------------------------
# ACA HAGO UN GRAFICO DE BURBUJA DEL APORTE DE CADA UNA DE LAS SSP Y DEL ESPECTRO SINTETICO RESULTANTE
# ------------------------------------------------------------------------------------------------------------------------------
x = np.log10(df1["age_j(yr)"])
x = round(x, 2)
y = df1["Z_j"]
aporte = df1["x_j(%)"]
porcen = [
    f"{valor:.1f}%" for valor in aporte
]  # esto es para que sobre la burjuja aparesca el aporte en forma de porcentaje

fig.add_scatter(
    x=x,
    y=y,
    mode="markers",
    marker=dict(size=30 + aporte, color="blue"),
    name="Aporte de SSP",
)  # SSP
fig.add_scatter(
    x=[edad],
    y=[met],
    mode="markers",
    marker=dict(size=20, color="red"),
    name="Promedio",
)  # Sintetico
fig.add_trace(
    go.Scatter(
        x=x,
        y=y,
        mode="text",
        text=porcen,
        textposition="middle center",
        name=" ",
    )
)  # texto sobre burbujas

# ------------------------------------------------------------------------------------------------------------------------------
# ACA AÑADO UNA NOTACION AL GRAFICO DE LA EDAD,METALICIDA Y CHI2 DEL ESPECTRO RESULTANTE
# ------------------------------------------------------------------------------------------------------------------------------
fig.add_annotation(
    xref="paper",
    yref="paper",
    x=1,
    y=1,
    text="Espectro sintetico: "
    + "Log(Edad) = "
    + f"{edad}"
    + " yr"
    + ", "
    + "Z = "
    + f"{met}"
    + ", "
    + "\u03C7\u00B2 = "
    + f"{round(df.iloc[0,0],2)}"
    + ", "
    + "Av = "
    + f"{round(df.iloc[3,0],2)}"
    + ", "
    + "V0 = "
    + f"{round(df.iloc[1,0],2)}"
    + " Km/s"
    + ", "
    + "Vd = "
    + f"{round(df.iloc[2,0],2)}"
    + " Km/s",
    showarrow=False,
    font=dict(size=18),
)
# -------------------------------------------------------------------------------------------------------------------------------
# ACA CONFIGURO LOS DATALLES DEL GRAFICO
# -------------------------------------------------------------------------------------------------------------------------------
fig.update_layout(
    title="Grafico de burbuja del archivo " + f"{nombre_archivo}",
    title_font=dict(size=22),
)
fig.update_xaxes(title_text="Edad (yr)", title_font=dict(size=18))
fig.update_yaxes(title_text="Abundacia", title_font=dict(size=18))


pyo.plot(fig)  # si pongo fig.show() me lo muestra en terminal

'temp-plot.html'

In [11]:
fig = go.Figure()
# -------------------------------------------------------------------------------------------------------------------------------
# ACA GRAFICO EL ESPECTRO OBSERVADO, EL SINTETICO Y LA DIFERENCIA ENTRE AMBOS
# -------------------------------------------------------------------------------------------------------------------------------
fig.add_scatter(x=df2["l_obs"], y=df2["f_obs"], name="observado")
fig.add_scatter(x=df2["l_obs"], y=df2["f_syn"], name="modelo")
fig.add_scatter(
    x=df2["l_obs"],
    y=(df2["f_obs"] - df2["f_syn"]),
    name="residuos",
    line=dict(width=0.7),
)
# -------------------------------------------------------------------------------------------------------------------------------
# ACA CALCULO LA DESVIACION ESTANDAR DE LA DIFERENCIA ENTRE EL MODELO Y LOS DATOS
# -------------------------------------------------------------------------------------------------------------------------------
sigma = np.std(df2["f_obs"] - df2["f_syn"])
# -------------------------------------------------------------------------------------------------------------------------------
# ACA GRAFICO LAS RECTAS DEL RESIDUO
# -------------------------------------------------------------------------------------------------------------------------------
fig.add_scatter(
    x=df2["l_obs"],
    y=[0] * len(df2["l_obs"]),
    name="residuo 0",
    line=dict(color="black", width=1, dash="dash"),
)
# fig.add_scatter( x = df2['l_obs'], y = [3*sigma] * len(df2['l_obs']), name = '+3 sigmas', line = dict(color = 'black',width = 1,dash = 'dash'))
# fig.add_scatter( x = df2['l_obs'], y = [-3*sigma] * len(df2['l_obs']), name ='-3 sigmas', line = dict(color = 'black',width = 1,dash = 'dash'))
# -------------------------------------------------------------------------------------------------------------------------------
# ACA AÑADO LAS ANOTACIONES AL GRAFICO
# -------------------------------------------------------------------------------------------------------------------------------
fig.add_annotation(
    xref="paper",
    yref="paper",
    x=1,
    y=1,
    text="Espectro sintetico: "
    + "Log(Edad) = "
    + f"{edad}"
    + " yr"
    + ", "
    + "Z = "
    + f"{met}"
    + ", "
    + "\u03C7\u00B2 = "
    + f"{round(df.iloc[0,0],2)}"
    + ", "
    + "Av = "
    + f"{round(df.iloc[3,0],2)}"
    + ", "
    + "V0 = "
    + f"{round(df.iloc[1,0],2)}"
    + " Km/s"
    + ", "
    + "Vd = "
    + f"{round(df.iloc[2,0],2)}"
    + " Km/s",
    showarrow=False,
    font=dict(size=18),
)

# -------------------------------------------------------------------------------------------------------------------------------
# ACA CONFIGURO LOS DATALLES DEL GRAFICO
# -------------------------------------------------------------------------------------------------------------------------------
fig.update_layout(
    title="Resuldatos de StarLight del archivo " + f"{nombre_archivo}",
    title_font=dict(size=22),
)
# fig.update_layout(legend=dict(x=0, y=1, bgcolor='rgba(255, 255, 255, 0.5)', bordercolor='gray', borderwidth=1))
fig.update_xaxes(title_text="Lambda", title_font=dict(size=18))
fig.update_yaxes(title_text="Flujo", title_font=dict(size=18))

# fig.show()
pyo.plot(fig)  # si pongo fig.show() me lo muestra en terminal

'temp-plot.html'